# Several implementations of tokenizers, text processing and indexing

In [1]:
import nltk
import pattern.en
import langdetect as ld
import string
from collections import defaultdict, Counter
import numpy as np

In [ ]:
class Tokenizer(object):
    
    def __init__(self, preserve_case=True):
        self.tweet = nltk.tokenize.TweetTokenizer(preserve_case=preserve_case)
    
    @staticmethod
    def lang(doc):
        return ld.detect(doc)
    
    @staticmethod
    def remove_punctuation(tokens, special_chars=None):
        p = string.punctuation
        if special_chars is not None:
            p += "".join(special_chars)
        return [x for x in tokens if x not in p]
    
    def tweet_tokenizer(self, doc):
        return self.tweet.tokenize(doc)

class MIndex(defaultdict):
    
    def __init__(self):
        super(MIndex, self).__init__(lambda: [])
        self.docs = set()
    
    def boolean(self, doc_id, tokens):
        self.docs.add(doc_id)
        for token in set(tokens):
            self[token].append(doc_id)
    
    def boolean_to_matrix(self):
        features = list(self.keys())
        docs = list(self.docs)
        M = np.zeros((len(docs), len(features)))
        for token, posting in self.items():
            for doc in posting:
                ti, di = features.index(token), docs.index(doc)
                M[di][ti] = 1
        return M > 0, features, docs